In [ ]:
!pip install torch_geometric

In [ ]:
!pip install torch-geometric-temporal

In [2]:
import torch
import torch.nn as nn
from torch_geometric.utils import to_dense_adj, dense_to_sparse
from torch_geometric.nn.conv import MessagePassing
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.dataset import PemsBayDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split
from tqdm import tqdm
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cpu


### DConvRNN Model Implementation
##### Paper: https://arxiv.org/abs/1707.01926

In [3]:
class DCRLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, C):
      super(DCRLayer, self).__init__(aggr='add', flow='source_to_target')
      self.in_channels = in_channels
      self.out_channels = out_channels
      self.C = C
      self.weight = torch.nn.Parameter(torch.Tensor(2, self.C, self.in_channels, self.out_channels)).to(device)
      torch.nn.init.xavier_uniform_(self.weight)
      self.bias = torch.nn.Parameter(torch.zeros(out_channels)).to(device)

    def message(self, x_i, norm):
      val = x_i * norm.view(-1, 1)
      return val

    def forward(self, input, edge_index, edge_weight):
        i, j = edge_index
        matrix = to_dense_adj(edge_index=edge_index, edge_attr=edge_weight).to(device)
        matrix = torch.reshape(input=matrix, shape=(matrix.shape[1], matrix.shape[2])).to(device)
        ones = torch.ones((matrix.shape[0], 1)).to(device)
        out_degree = torch.matmul(matrix, ones).flatten().to(device)
        out_degree_inverse = torch.reciprocal(out_degree)
        out_norm = out_degree_inverse[i]

        in_degree = torch.matmul(ones.T, matrix).flatten()
        in_degree_inverse = torch.reciprocal(in_degree)
        in_norm = in_degree_inverse[i]

        edge_index_reverse, _ = dense_to_sparse(matrix.T)

        result = torch.matmul(input, (self.weight[0])[0]) + torch.matmul(input, (self.weight[1])[0])
        if self.weight.shape[1] > 1:
          p_0 = self.propagate(edge_index, x=input, norm=out_norm, size=None)
          p_1 = self.propagate(edge_index_reverse, x=input, norm=in_norm, size=None)
          result = result + torch.matmul(p_0, (self.weight[0])[1]) + torch.matmul(p_1, (self.weight[1])[1])

        X_0 = input
        X_1 = input
        for x in range(2, self.weight.shape[1]):
            p_x_0 = self.propagate(edge_index, x=p_0, norm=out_norm, size=None)
            p_x_0 = 2.0 * p_x_0 - X_0
            p_x_1 = self.propagate(edge_index_reverse, x=p_1, norm=in_norm, size=None)
            p_x_1 = 2.0 * p_x_1 - X_0
            result = result + torch.matmul(p_x_0, (self.weight[0])[x]) + torch.matmul(p_x_1, (self.weight[1])[x])
            X_0 = X_1
            p_0 = p_x_0
            p_1 = p_x_1

        result += self.bias
        return result

In [4]:
class DCGRU(nn.Module):
  def __init__(self, in_channels, out_channels, C):
    super(DCGRU, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.C = C

    # Update
    self.conv_layer_1 = DCRLayer(in_channels=self.in_channels + self.out_channels, out_channels=self.out_channels, C=self.C).to(device)
    self.sigmoid_1 = nn.Sigmoid()
    # Reset
    self.conv_layer_2 = DCRLayer(in_channels=self.in_channels + self.out_channels, out_channels=self.out_channels, C=self.C).to(device)
    self.sigmoid_2 = nn.Sigmoid()
    # Candidate
    self.conv_layer_3 = DCRLayer(in_channels=self.in_channels + self.out_channels, out_channels=self.out_channels, C=self.C).to(device)
    self.sigmoid_3 = nn.Sigmoid()

  def forward(self, input, edge_weight, edge_index, hidden=None):
    # Update
    outs_1 = torch.cat([input, hidden], dim=-1).to(device)
    outs_1 = self.conv_layer_1(outs_1, edge_index, edge_weight).to(device)
    outs_1 = self.sigmoid_1(outs_1).to(device)

    # Reset
    outs_2 = torch.cat([input, hidden], dim=-1).to(device)
    outs_2 = self.conv_layer_2(outs_2, edge_index, edge_weight).to(device)
    outs_2 = self.sigmoid_2(outs_2).to(device)

    # Candidate
    hidden_3 = hidden * outs_2
    outs_3 = torch.cat([input, hidden_3], dim=-1).to(device)
    outs_3 = self.conv_layer_3(outs_3, edge_index, edge_weight).to(device)
    outs_3 = self.sigmoid_3(outs_3).to(device)

    result = outs_1 * hidden + (1 - outs_1) * outs_3
    return result.to(device)



In [5]:
class DCEncoder(nn.Module):
  def __init__(self, in_channels, out_channels, C):
    super(DCEncoder, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.C = C
    self.layer_1 = DCGRU(in_channels=self.in_channels, out_channels=self.out_channels, C=self.C).to(device)
    self.layer_2 = DCGRU(in_channels=self.out_channels, out_channels=self.out_channels, C=self.C).to(device)

  def forward(self, input, edge_weight, edge_index, hidden_states):
    hiddens = []
    outs = self.layer_1(input, edge_weight, edge_index, hidden=hidden_states[0]).to(device)
    hiddens.append(outs)
    outs = self.layer_2(outs, edge_weight, edge_index, hidden=hidden_states[1]).to(device)
    hiddens.append(outs)
    return outs, hiddens


In [6]:
class DCDecoder(nn.Module):
  def __init__(self, in_channels, out_channels, output_dim, C):
    super(DCDecoder, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.output_dim = output_dim
    self.C = C
    self.layer_1 = DCGRU(in_channels=self.in_channels, out_channels=self.out_channels, C=self.C).to(device)
    self.layer_2 = DCGRU(in_channels=self.out_channels, out_channels=self.out_channels, C=self.C).to(device)
    self.fc_layer = nn.Linear(self.out_channels, self.output_dim).to(device)

  def forward(self, input, edge_weight, edge_index, hidden_states):
    hiddens = []
    outs = self.layer_1(input.to(device), edge_weight.to(device), edge_index.to(device), hidden=hidden_states[0].to(device)).to(device)
    hiddens.append(outs)
    outs = self.layer_2(outs.to(device), edge_weight.to(device), edge_index.to(device), hidden=hidden_states[1].to(device)).to(device)
    hiddens.append(outs)
    outs = self.fc_layer(outs.to(device)).to(device)
    return outs.to(device), hiddens
    

In [7]:
class DConvRNN(nn.Module):
  def __init__(self, in_channels, out_channels, output_dim, C):
    super(DConvRNN, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.output_dim = output_dim
    self.C = C
    self.encoder = DCEncoder(self.in_channels, self.out_channels, self.C)
    self.decoder = DCDecoder(self.in_channels, self.out_channels, self.output_dim, self.C)

  def encode(self, input, edge_weight, edge_index):
    hidden_states = [
      torch.zeros(input.shape[0], self.out_channels).to(device),
      torch.zeros(input.shape[0], self.out_channels).to(device)
    ]
    _, hidden_states = self.encoder(input, edge_weight, edge_index, hidden_states)
    return hidden_states

  def decode(self, input, edge_weight, edge_index, hidden_states):
    outputs, hidden_states = self.decoder(input.to(device), edge_weight.to(device), edge_index.to(device), hidden_states)
    return outputs

  def forward(self, input, predictions, edge_weight, edge_index):
    hidden = self.encode(input, edge_weight, edge_index)
    outputs = self.decode(predictions.to(device), edge_weight.to(device), edge_index.to(device), hidden)
    return outputs


#### Model Architecture

In [8]:
dcrnn = DConvRNN(in_channels=12, out_channels=32, output_dim=12, C=2)
print(dcrnn)

DConvRNN(
  (encoder): DCEncoder(
    (layer_1): DCGRU(
      (conv_layer_1): DCRLayer(44, 32)
      (sigmoid_1): Sigmoid()
      (conv_layer_2): DCRLayer(44, 32)
      (sigmoid_2): Sigmoid()
      (conv_layer_3): DCRLayer(44, 32)
      (sigmoid_3): Sigmoid()
    )
    (layer_2): DCGRU(
      (conv_layer_1): DCRLayer(64, 32)
      (sigmoid_1): Sigmoid()
      (conv_layer_2): DCRLayer(64, 32)
      (sigmoid_2): Sigmoid()
      (conv_layer_3): DCRLayer(64, 32)
      (sigmoid_3): Sigmoid()
    )
  )
  (decoder): DCDecoder(
    (layer_1): DCGRU(
      (conv_layer_1): DCRLayer(44, 32)
      (sigmoid_1): Sigmoid()
      (conv_layer_2): DCRLayer(44, 32)
      (sigmoid_2): Sigmoid()
      (conv_layer_3): DCRLayer(44, 32)
      (sigmoid_3): Sigmoid()
    )
    (layer_2): DCGRU(
      (conv_layer_1): DCRLayer(64, 32)
      (sigmoid_1): Sigmoid()
      (conv_layer_2): DCRLayer(64, 32)
      (sigmoid_2): Sigmoid()
      (conv_layer_3): DCRLayer(64, 32)
      (sigmoid_3): Sigmoid()
    )
    (fc_la

### Build Datasets

Metr LA Dataset

In [7]:
metr_la_loader = METRLADatasetLoader()
metr_la_dataset = metr_la_loader.get_dataset()
metr_la_train_dataset, metr_la_val_dataset = temporal_signal_split(metr_la_dataset, train_ratio=0.7)
metr_la_val_dataset, metr_la_test_dataset = temporal_signal_split(metr_la_val_dataset, train_ratio=0.5)

Pems Bay Dataset

In [8]:
pems_bay_loader = PemsBayDatasetLoader()
pems_bay_dataset = pems_bay_loader.get_dataset()
pems_bay_train_dataset, pems_bay_val_dataset = temporal_signal_split(pems_bay_dataset, train_ratio=0.7)
pems_bay_val_dataset, pems_bay_test_dataset = temporal_signal_split(pems_bay_val_dataset, train_ratio=0.5)

### Train and Evaluate Model

In [13]:
def train_model(model, optimizer, loss_fn, num_epochs, train_dataset_loader, val_dataset_loader, is_la=True):
  model.train()
  for epoch in range(num_epochs):
    print("Epoch: " + str(epoch + 1) + " / " + str(num_epochs))
    total_loss = 0
    total_len = 0
    rmse = 0
    mae = 0
    mape = 0
    for _, snapshot in tqdm(enumerate(train_dataset_loader)):
      optimizer.zero_grad()
      input = snapshot.x[:, 0, :]
      target = snapshot.y.to(device) if is_la else snapshot.y[:, 0, :].to(device)
      edge_weight = snapshot.edge_attr.to(device)
      edge_index = snapshot.edge_index.to(device)
      predictions = model(input.to(device), target.to(device), edge_weight.to(device), edge_index.to(device)).to(device)
      
      loss = loss_fn(predictions, target)
      loss.backward()
      total_loss += loss.item()
      optimizer.step()

      rmse += mean_squared_error(target.detach().cpu().numpy(), predictions.detach().cpu().numpy(), squared=False)
      mae += mean_absolute_error(target.detach().cpu().numpy(), predictions.detach().cpu().numpy())
      mape += mean_absolute_percentage_error(target.detach().cpu().numpy(), predictions.detach().cpu().numpy())
      total_len += 1
    
    epoch_loss = total_loss / total_len
    rmse /= total_len
    mae /= total_len
    mape /= total_len
    print("Epoch {} train Loss: {:.4f}, train RMSE: {:.4f}, train MAE: {:.4f}, train MAPE: {:.4f}".format(epoch, epoch_loss, rmse, mae, mape))
    val_loss, val_rmse, val_mae, val_mape = eval_model(model, loss_fn, val_dataset_loader, is_la)
    print("Epoch {} val Loss: {:.4f}, val RMSE: {:.4f}, val MAE: {:.4f}, val MAPE: {:.4f}".format(epoch, val_loss, val_rmse, val_mae, val_mape))
  

def eval_model(model, loss_fn, val_dataset_loader, is_la=True):
  model.eval()
  total_loss = 0
  total_len = 0
  rmse = 0
  mae = 0
  mape = 0
  with torch.no_grad():
    for _, snapshot in tqdm(enumerate(val_dataset_loader)):
      input = snapshot.x[:, 0, :]
      target = snapshot.y.to(device) if is_la else snapshot.y[:, 0, :].to(device)
      edge_weight = snapshot.edge_attr.to(device)
      edge_index = snapshot.edge_index.to(device)
      if total_len == 0:
        predictions = torch.randn_like(target).to(device)
      predictions = model(input.to(device), predictions.to(device), edge_weight.to(device), edge_index.to(device)).to(device)
      loss = loss_fn(predictions, target)
      total_loss += loss.item()

      rmse += mean_squared_error(target.detach().cpu().numpy(), predictions.detach().cpu().numpy(), squared=False)
      mae += mean_absolute_error(target.detach().cpu().numpy(), predictions.detach().cpu().numpy())
      mape += mean_absolute_percentage_error(target.detach().cpu().numpy(), predictions.detach().cpu().numpy())
      total_len += 1
    
    val_loss = total_loss / total_len
    rmse /= total_len
    mae /= total_len
    mape /= total_len
  return val_loss, rmse, mae, mape

### Train and Test LA Dataset

In [14]:
la_model = DConvRNN(in_channels=12, out_channels=32, output_dim=12, C=2)
optimizer = torch.optim.Adam(la_model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()
train_model(la_model, optimizer, loss_fn, 5, metr_la_train_dataset, metr_la_val_dataset)

Epoch: 1 / 5


23974it [14:24, 27.74it/s]


Epoch 0 train Loss: 0.0246, train RMSE: 0.1022, train MAE: 0.0683, train MAPE: 0.4132


5137it [01:48, 47.26it/s]


Epoch 0 val Loss: 3.9307, val RMSE: 1.9685, val MAE: 1.5856, val MAPE: 7.4804
Epoch: 2 / 5


...

Epoch 3 val Loss: 4.9988, val RMSE: 2.2146, val MAE: 1.8099, val MAPE: 9.1985
Epoch: 5 / 5


23974it [15:20, 26.05it/s]


Epoch 4 train Loss: 0.0024, train RMSE: 0.0275, train MAE: 0.0188, train MAPE: 0.0996


5137it [01:48, 47.32it/s]

Epoch 4 val Loss: 4.6748, val RMSE: 2.1446, val MAE: 1.7085, val MAPE: 8.0834


In [15]:
test_loss, test_rmse, test_mae, test_mape = eval_model(la_model, loss_fn, metr_la_test_dataset)
print("Test Loss: {:.4f}, test RMSE: {:.4f}, test MAE: {:.4f}, test MAPE: {:.4f}".format(test_loss, test_rmse, test_mae, test_mape))

5138it [01:34, 54.64it/s]

Test Loss: 4.6412, test RMSE: 2.1410, test MAE: 1.6952, test MAPE: 8.0144


### Train and Test Pems Bay Dataset

In [16]:
pems_model = DConvRNN(in_channels=12, out_channels=32, output_dim=12, C=2)
optimizer = torch.optim.Adam(pems_model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()
train_model(pems_model, optimizer, loss_fn, 5, pems_bay_train_dataset, pems_bay_val_dataset, is_la=False)

Epoch: 1 / 5


36457it [34:26, 17.64it/s]


Epoch 0 train Loss: 0.0073, train RMSE: 0.0611, train MAE: 0.0373, train MAPE: 0.2154


7812it [04:06, 31.67it/s]


Epoch 0 val Loss: 3.4305, val RMSE: 1.7984, val MAE: 1.4061, val MAPE: 8.2047
Epoch: 2 / 5


...

Epoch 3 val Loss: 6.8991, val RMSE: 2.5876, val MAE: 1.9976, val MAPE: 12.2093
Epoch: 5 / 5


36457it [31:14, 19.44it/s]


Epoch 4 train Loss: 0.0004, train RMSE: 0.0124, train MAE: 0.0076, train MAPE: 0.0392


7812it [03:47, 34.28it/s]


Epoch 4 val Loss: 7.8628, val RMSE: 2.7498, val MAE: 2.1430, val MAPE: 12.3037


In [18]:
test_loss, test_rmse, test_mae, test_mape = eval_model(pems_model, loss_fn, pems_bay_test_dataset, is_la=False)
print("Test Loss: {:.4f}, test RMSE: {:.4f}, test MAE: {:.4f}, test MAPE: {:.4f}".format(test_loss, test_rmse, test_mae, test_mape))

7813it [03:23, 38.48it/s]

Test Loss: 7.5590, test RMSE: 2.7034, test MAE: 2.1133, test MAPE: 13.3226
